# Model walk-through

### Load required packages

In [1]:
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following object is masked _by_ '.GlobalEnv':

    count

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



### Create and set directory for saved files

In [ ]:
dir.create(path=paste(getwd(), paste("para_set", para_set, sep="_"), paste("model_run", r, sep="_"), sep="/"), recursive=T)
#directory is created using the para_set number read in from the parameter script and parameters csv and the model_run number from the commander script

setwd(paste(getwd(), paste("para_set", para_set, sep="_"), paste("model_run", r, sep="_"), sep="/"))
#directory is set relative to the current working directory, which is set in the commander script

### A blank dataframe is created to hold one value per generation and column names are provided

In [ ]:
output<-as.data.frame(matrix(nrow=no_gen, ncol=11))
#no_gen is taken from the parameters script
names(output)<-c("gen","FTmean","FTvar","FTh2","IBmean","pA","pB","pC","FisA","FisB","FisC")

### A blank dataframe is made to hold information about genetic neighbourhood size

In [ ]:
sum.N<-as.data.frame(matrix(nrow=no_gen, ncol=3))

## The parental generation is created

### Step 1) Create a table to hold parental information

In [ ]:
parents<-as.data.frame(matrix(nrow=pop_size, ncol=(10+2*FT_loci)+4+2*neut_loci))
#2 columns are needed for each locus so that each allele per locus can be stored

names(parents)[1:10]<-c("Mother","Father","IB_coef","FLday","A1","A2","B1","B2","C1","C2")
#A, B and C are three additional neutral loci used primarily for heatmap purposes
#A can also be set to be under selection using the parameter .csv

for (n in 1:FT_loci)
{
  names(parents)[11+2*(n-1)]<-paste("loc",n,"a",sep="")
  names(parents)[12+2*(n-1)]<-paste("loc",n,"b",sep="")
}

names(parents)[(ncol(parents)-3): ncol(parents)]<-c("FTnoise","position","X_pos","Y_pos")
for (j in 1:neut_loci)
{
  names(parents)[11+(2*FT_loci)+(2*(j-1))]<-paste("neut",j,"a",sep="")
  names(parents)[12+(2*FT_loci)+(2*(j-1))]<-paste("neut",j,"b",sep="")
}

#name assignments are set up so column positions are relative to the number of neutral loci specified

### Step 2) Generate flowering time onsets

#### 2a) Give each individual its flowering time genotype (2 alleles)

In [ ]:
for (n in 1:FT_loci){
  parents[,11+2*(n-1)]<-sample(FT_alleles, size=pop_size, replace=T)
  parents[,12+2*(n-1)]<-sample(FT_alleles, size=pop_size, replace=T)
}

#potential alleles are drawn from FT_alleles, which are specified in the parameters script
#to compare to neutral loci, best to make each FT locus biallelic

#### 2b) Add environmental variance to flowering time

In [ ]:
parents$FTnoise<-rnorm(n=pop_size, mean=0, sd=fl_noise)

#### 2c) Add genetic and environmental components to a positive baseline (mean flowering day) to get individual flowering days

In [ ]:
parents$FLday<-floor(fl_mean + rowSums(parents[,11:(10+2*FT_loci)])+parents$FTnoise)

### Step 3) Assign genotypes at neutral loci

#### 3a) Give each individual a genotype at the A, B and C loci

In [ ]:
parents$A1<-sample(c("A","a"), size=pop_size, prob=c(freqA, 1-freqA), replace=T)
parents$A2<-sample(c("A","a"), size=pop_size, prob=c(freqA, 1-freqA), replace=T)

parents$B1<-sample(c("B","b"), size=pop_size, prob=c(freqB, 1-freqB), replace=T)
parents$B2<-sample(c("B","b"), size=pop_size, prob=c(freqB, 1-freqB), replace=T)

parents$C1<-sample(c("C","c"), size=pop_size, prob=c(freqC, 1-freqC), replace=T)
parents$C2<-sample(c("C","c"), size=pop_size, prob=c(freqC, 1-freqC), replace=T)

#starts population at HWE
#allele frequencies are typically starting at 0.5 (can be adjusted to alter starting conditions)

#### 3b) Give each individual a genotype at its remaining neutral loci

In [ ]:
freqD<-0.5

for (j in 1:neut_loci)
    {
      parents[,11+(2*FT_loci)+(2*(j-1))]<-sample(c("D","d"), size=pop_size, prob=c(freqD, 1-freqD), replace=T)
      parents[,12+(2*FT_loci)+(2*(j-1))]<-sample(c("D","d"), size=pop_size, prob=c(freqD, 1-freqD), replace=T)
    }

### Step 4) Give each individual a position in space

#### 4a) From matrix of available positions in space, develop list of positions from which to sample

In [ ]:
for (y in 1:ncol(space_20by20_sq_grid)) {
    if (y == 1) {
        pos_vector<-c(as.character(space_20by20_sq_grid[,y]))
    } else {
        pos_vector<-c(pos_vector, as.character(space_20by20_sq_grid[,y]))
    }
}

#### 4b) Give everyone a position from the set of possible positions, and then extract x and y coordinates

In [ ]:
parents$position<-(pos_vector)
parents$X_pos<-sapply(X=parents$position, FUN=function(X) {
  as.numeric(strsplit(X, split=",")[[1]][1])
})
parents$Y_pos<-sapply(X=parents$position, FUN=function(X) {
  as.numeric(strsplit(X, split=",")[[1]][2])
})

#sapply takes each item in the dataframe and performs the function on it individually (avoids looping)
#strsplit takes each item in the parents$position column, splits it at the comma and returns the first item if [[1]][1] ro second if [[1]][2]
#the output of strsplit is then assigned to new columns as the X and Y positions

### Step 5) Calculate coancestry and inbreeding coefficients of parents

#### 5a) Create coancestry matrix

In [ ]:
coancestry_p<-matrix(nrow=pop_size, ncol=pop_size)

#### 5b) Set coancestry values (all 0 except along diagonal which is 0.5)

In [ ]:
coancestry_p[,]<-0
for (i in 1:pop_size) {
  coancestry_p[i,i]<-0.5
}

#### 5c) Set inbreeding coefficient as zero because it's generation 1

In [ ]:
parents$IB_coef<-0

## Run offspring generations (2 through no_gen)

##### Start loop over generations; silenced in this walk-through because the code inside the loop is interrupted by annotations

In [ ]:
for (g in 1:no_gen) {

### Step 1) Store generation g conditions into the output table (one value per gen)

In [ ]:
output$gen[g]<-g
output$FTmean[g]<-mean(parents$FLday)
output$FTvar[g]<-var(parents$FLday)
output$FTh2[g]<-var(rowSums(parents[,11:(10+2*FT_loci)]))/var(parents$FLday)
output$IBmean[g]<-mean(parents$IB_coef)
output$pA[g]<-length(which(c(parents$A1, parents$A2)=="A"))/(2*pop_size)
output$pB[g]<-length(which(c(parents$B1, parents$B2)=="B"))/(2*pop_size)
output$pC[g]<-length(which(c(parents$C1, parents$C2)=="C"))/(2*pop_size)

output$FisA[g]<-((2*output$pA[g]*(1-output$pA[g])) - (length(which(Ageno_tab[,g]=="aA"))/pop_size))/(2*output$pA[g]*(1-output$pA[g]))
output$FisB[g]<-((2*output$pB[g]*(1-output$pB[g])) - (length(which(Bgeno_tab[,g]=="bB"))/pop_size))/(2*output$pB[g]*(1-output$pB[g]))
output$FisC[g]<-((2*output$pC[g]*(1-output$pC[g])) - (length(which(Cgeno_tab[,g]=="cC"))/pop_size))/(2*output$pC[g]*(1-output$pC[g]))

### Step 2) Fill in flowering schedule

#### 2a) Set days of flowering schedule

In [ ]:
days<-c(min(parents$FLday):(max(parents$FLday)+(duration-1)))

#### 2b) Fill in flowering schedule using parental generation abundances

In [ ]:
flowers<-as.data.frame(matrix(nrow=pop_size, ncol=length(days)))
#flowers matrix is as wide as there are days in the flowering season, so an individual's flowers on each day are recorded
names(flowers)<-paste("d", days, sep="")

for (i in 1:pop_size){
    flowers[i, which(days==parents$FLday[i]):(which(days==parents$FLday[i])+duration-1)]<-fl_seq
} #applies flowering schedule from parameters script to this range of days for an individual

flowers[is.na(flowers)]<-0 #removes NA days

#### 2c) Account for inbreeding depression is applicable

In [ ]:
if (LA_inbreeding_flowers == "yes") {
    for (i in 1:nrow(flowers)){
      flowers[i,]<-flowers[i,]*(fl_ID_weight)*(1-parents$ID_coef[i])
    }
}
#lowers fitness of individuals if they are inbred by lowering their flower output

### Step 3) Create mating opportunity matrix based on phenology

In [ ]:
if (phenology== "yes") {
    #Set maternal matrix as the proportion of flowers made by a single individual relative to those made by the total population (both over entire season)
    mat<-as.matrix(flowers/sum(flowers))
    #dimensions are pop_size x length(days)
    #mat flower adjusted for total flowers made by the whole population over the entire flowering season

    #Set paternal matrix as the proportion of flowers made by a single individual relative to the entire population
    pat<-matrix(nrow=pop_size, ncol=length(days))

    #pat flower are adjusted for other flowers open on each day of an individual's flowering peiod
    for (t in 1:ncol(pat))	{	
      pat[,t]<-flowers[,t]/sum(flowers[,t])
    }
    #t is a day in the flowering season, because the columns in flower are days

    #Fix pat matrix to deal with days where NO flowers are produced in the population (this removes NaN)
    if (any(colSums(flowers)==0)){
      pat[,which(colSums(flowers)==0)]<-0
    }

    #Place moms as rows and dads as columns
    mat_opp<-mat %*% t(pat)

} else if (phenology=="no") {
    mat_opp<-matrix(nrow=pop_size, ncol=pop_size)
    mat_opp[,]<-1/(pop_size^2)
} #if phenology is not affecting mating probabilities, then everyone has equal opportunity so prob for individuals i and j is just 1/n

### Step 4) Develop a compatibility matrix bsed on self-incompatibility alleles

In [ ]:
if (compatibility == "yes") {
    S_allele_matching<-matrix(nrow=pop_size, ncol=pop_size)
    for (m in 1:pop_size) {
        for (p in 1:pop_size) {
            S_allele_matching[m, p]<- if (m == p) {0} else if (m != p) {1}
        } #assuming an infinite number of SI alleles, such that selfing cannot occur but individual i can mate with any other individual in the population (technically)
    }
} else if (compatibility=="no") {
    S_allele_matching<-matrix(nrow=pop_size, ncol=pop_size)
    S_allele_matching[,]<-1
} #if SI is not included, then everyone just gets a 1

### Step 5) Recalibrate the mating opportunity matrix to account for self-incompatibility

In [ ]:
mat_opp_adj<-mat_opp * S_allele_matching

#Re-adjust everything so that matrix sums to 1 (these need to be probability matrices)
mat_opp_adj<-mat_opp_adj/sum(mat_opp_adj)

### Step 6) Develop pairwise geographic space matrix and run through pollen dispersal function

In [ ]:
if (torus == "no") { #a torus is used to minimise edge effects; however, not biologically realistic
  distance1<-matrix(nrow=pop_size, ncol=pop_size)
  distance2<-matrix(nrow=pop_size, ncol=pop_size)
    
  for (m in 1:pop_size) {
    for (p in 1:pop_size) {
      distance1[m,p]<-sqrt((parents$X_pos[m]-parents$X_pos[p])^2 + (parents$Y_pos[m] - parents$Y_pos[p])^2)	
    } #distance is just the Euclidean distance between two points
  }
  distance1<-distance1*m_per_unit
  distance2[,]<-0 #when not on a torus, distance is not considered in the second direction

  #6a1) Run through pollen dispersal function
  if (space_pollen=="yes"){ #does pollen have limited dispersal?
    distance_for_mating1<-distance1
    distance_for_mating2<-distance2
  } else if (space_pollen=="no") {
    distance_for_mating1<-matrix(nrow=pop_size, ncol=pop_size)
    distance_for_mating2<-matrix(nrow=pop_size, ncol=pop_size)
    distance_for_mating1[,]<-1
    distance_for_mating2[,]<-0
  }

  pollen_dispersal1<-(exp(-(seed_lambda*distance_for_mating1))) #if space_pollen is no, everyone gets the same value
  pollen_dispersal2<-0 #everyone gets the same value here as well

} else if (torus == "yes") {
    distance1<-matrix(nrow=pop_size, ncol=pop_size)
    distance2<-matrix(nrow=pop_size, ncol=pop_size)
    for (m in 1:pop_size) {
        for (p in 1:pop_size) {
            distance1[m,p]<-sqrt((parents$X_pos[m]-parents$X_pos[p])^2 + (parents$Y_pos[m] - parents$Y_pos[p])^2)
            distance2[m,p]<-sqrt(((sqrt(pop_size)+1)-abs(parents$X_pos[m]-parents$X_pos[p]))^2 + ((sqrt(pop_size)+1)-abs(parents$Y_pos[m]-parents$Y_pos[p]))^2)
        } #the second distance is between the same two points but 'wraps' around the other side of the grid
    }
    distance1<-distance1*m_per_unit #distance is adjusted by spacing between individuals
    distance2<-distance2*m_per_unit

    #6a1) Run through pollen dispersal function
    if (space_pollen=="yes"){
        distance_for_mating1<-distance1
        distance_for_mating2<-distance2
    } else if (space_pollen=="no") {
        distance_for_mating1<-matrix(nrow=pop_size, ncol=pop_size)
        distance_for_mating1[,]<-1
        distance_for_mating2<-matrix(nrow=pop_size, ncol=pop_size)
        distance_for_mating2[,]<-1
    }
    pollen_dispersal1<-(exp(-(seed_lambda*distance_for_mating1)))
    pollen_dispersal2<-(exp(-(seed_lambda*distance_for_mating2)))
} #end of if torus "yes"

### Step 7) Recalibrate mating opportunity matrix to account for space

In [ ]:
mat_opp_adj2<-mat_opp_adj * (pollen_dispersal1 + pollen_dispersal2) #because pollen can be dispersed in either direction 1 OR direction 2, the probabilities are additive
mat_opp_adj2<-mat_opp_adj2/sum(mat_opp_adj2) #readjusts the matrix to a probability

### Step 8) Calculate relative fitness based on the A locus and flowering time

In [ ]:
parents$zFLday<-(parents$FLday-mean(parents$FLday))/sd(parents$FLday) #deviation from mean flowering day
parents$fitness<-NA
for (i in 1:pop_size){
    genotype<-paste(parents$A1[i], parents$A2[i], sep="") 
    parents$fitness[i]<-if (genotype=="aa") {
        1 + selection_aa*parents$zFLday[i] } else if (genotype=="AA"){1 + selection_AA*parents$zFLday[i]} else {1 + selection_Aa*parents$zFLday[i]}
} #sets fitness as values around 1 based on A locus and in what direction of the mean flowering day the individual sits (will be relative because z is calulated relative to population)

#### 8a) Correct for fitness values that are negative

In [ ]:
parents$fitness[which(parents$fitness<0)]<-0

#### 8b) Adjust fitness according to inbreeding

In [ ]:
if (LA_inbreeding_fecundity == "yes") {
    parents$fitness<-parents$fitness * (1-parents$IB_coef)
}

### Step 9) Generation offspring
##### The probability of choosing a given mom depends on her fitness and her distance from the xy position being filled
##### The dad's identity depends on the adjusted mating matrix
##### If a chosen mom has no available crosses, another mom is chosen

#### 9a) Create offfspring dataframe and set column names

In [ ]:
offspring<-as.data.frame(matrix(nrow=pop_size, ncol=ncol(parents)))
names(offspring)<-names(parents)

#### 9b) Designate spatial locations, assuming each space will be occupied by a single offspring

In [ ]:
offspring$position<-parents$position
offspring$X_pos<-parents$X_pos
offspring$Y_pos<-parents$Y_pos

#### 9c) Start loop over offspring

In [4]:
for (o in 1:pop_size) {

    #Calculate dispersal of seeds to location of offspring[o] as a weight on relative fitness and distance
    if (space_seed == "yes"){
        seed_dispersal1<-(exp(-(seed_lambda*distance1[o,])))
        seed_dispersal<-seed_dispersal1
    } else if (space_seed == "no") {
        seed_dispersal1<-rep(1, times=pop_size)
        seed_dispersal<-seed_dispersal1
    } 
    seed_dispersal<-seed_dispersal1
    
    if ((space_seed == "yes") & (torus == "yes")) { 
      seed_dispersal2<-(exp(-(seed_lambda*distance2[o,])))
      seed_dispersal<-seed_dispersal1+seed_dispersal2
    } else if ((space_seed == "no") & (torus == "yes")) {
        seed_dispersal2<-rep(1, times=pop_size)
        seed_dispersal<-seed_dispersal1+seed_dispersal2
    }

    #Choose a mom based on relative fitness and chance of getting a seed to the specified location
        ##NOTE- Edge plants at an inherent disadvantage if a torus is not used.
    mom<-sample(c(1:pop_size), size=1, prob=(parents$fitness*seed_dispersal)/sum(parents$fitness*seed_dispersal))

    #If there are no sires for chosen mom, choose another mom
    while (rowSums(mat_opp_adj2)[mom]==0) {
        mom<-sample(c(1:pop_size), size=1, prob=(parents$fitness)/sum(parents$fitness))
    }

    #Choose sire for selected mom
    pat_prob<-mat_opp_adj2[mom,]/rowSums(mat_opp_adj2)[mom]
    dad<-sample(x=c(1:pop_size), size=1, prob=pat_prob)

    #9b5) Record mom, dad, and inbreeding coefficient
    offspring$Mother[o]<-mom
    offspring$Father[o]<-dad
    offspring$IB_coef[o]<-coancestry_p[mom,dad]

    #If early acting inbreeding-depression is in place, check to see if IB_coef is below threshold
    #If it's too high, then discard the seed and choose again
    if (EA_inbreeding == "yes") {
        while (offspring$IB_coef[o] > EA_id_threshold) {
            #Choose a new mom
            mom<-sample(c(1:pop_size), size=1, prob=(parents$fitness*seed_dispersal)/sum(parents$fitness*seed_dispersal))
            #If there are no sires for chosen mom, choose another.
            while (rowSums(mat_opp_adj2)[mom]==0) {
                mom<-sample(c(1:pop_size), size=1, prob=(parents$fitness)/sum(parents$fitness))
            }
        } #close while statement regarding threshold
    } #close if over whether early-acting ID in is place

    #A-locus inhertiance (one from mom, one from dad)
    offspring$A1[o]<-as.character(sample(x=parents[mom, 5:6], size=1))
    offspring$A2[o]<-as.character(sample(x=parents[dad, 5:6], size=1))

    #B-locus inhertiance (one from mom, one from dad)
    offspring$B1[o]<-as.character(sample(x=parents[mom, 7:8], size=1))
    offspring$B2[o]<-as.character(sample(x=parents[dad, 7:8], size=1))

    #C-locus inhertiance (one from mom, one from dad)
    offspring$C1[o]<-as.character(sample(x=parents[mom, 9:10], size=1))
    offspring$C2[o]<-as.character(sample(x=parents[dad, 9:10], size=1))

    #Mendellian inheritance of flowering time loci 
    for (n in 1:(FT_loci)) {
      offspring[o,11 + 2*(n-1)]<-sample(x=parents[mom,(11+2*(n-1)):(12 + 2*(n-1))], size=1)
      offspring[o,12 + 2*(n-1)]<-sample(x=parents[dad,(11+2*(n-1)):(12 + 2*(n-1))], size=1)
    } #close loop over the flowering time loci

    #Mutation at flowering time loci
      if (mut == "yes") {
          for (n in 1:(FT_loci)) {
            mutation<-sample(1:(1/mut_rate), size=1) #assuming mutation rate is for flowering time loci
            if (mutation == 1){
              effect<-sample((-1:1),size=1)
              parent_mut<-sample(1:2,size=1)
              if (parent_mut == 1) {
                  offspring[o,11+2*(n-1)]<-offspring[o,11+2*(n-1)]+effect
              } else {
                  offspring[o,12+2*(n-1)]<-offspring[o,12+2*(n-1)]+effect
              }
            }
         }
      }
    
    #Mendellian inheritance of remaining neutral loci
    for (j in 1:(neut_loci)) {
      offspring[o,11+(2*FT_loci)+(2*(j-1))]<-as.character(sample(x=parents[mom,(11+(2*FT_loci)+(2*(j-1))):(12+(2*FT_loci)+(2*(j-1)))], size=1))
      offspring[o,12+(2*FT_loci)+(2*(j-1))]<-as.character(sample(x=parents[dad,(11+(2*FT_loci)+(2*(j-1))):(12+(2*FT_loci)+(2*(j-1)))], size=1))
    } #close loop over remaining neutral loci
    
    #Mutation at neutral loci
      if (mut == "yes") {
          for (n in 1:(neut_loci)) {
            mutation<-sample(1:(1/mut_rate), size=1) #assuming mutation rate is for flowering time loci
            if (mutation == 1){
              effect<-sample(c("D","d"),size=1)
              parent_mut<-sample(1:2,size=1)
              if (parent_mut == 1) {
                  offspring[o,11+(2*FT_loci)+(2*(n-1))]<-effect
              } else {
                  offspring[o,12+(2*FT_loci)+(2*(n-1))]<-effect
              }
            }
         }
      }
} #close loop over offspring

ERROR: Error in parse(text = x, srcfile = src): <text>:98:15: unexpected symbol
97:               effect<-sample(c(,size=1)
98:               parent_mut
                  ^


#### 9d) Add environmental variance to flowering time, and sum flowering time loci columns to calculate flowering time

In [ ]:
offspring$FTnoise<-rnorm(n=pop_size, mean=0, sd=fl_noise)
offspring$FLday<-floor(fl_mean + rowSums(offspring[,11:(10+2*FT_loci)]) + offspring$FTnoise) 

### Step 10) Calculate coancestry in the new generation

In [ ]:
coancestry_o<-matrix(nrow=pop_size, ncol=pop_size)

for (i in 2:pop_size){
    for (j in 1:(i-1)){	
      Mi<-offspring$Mother[i]
      Mj<-offspring$Mother[j]
      Fi<-offspring$Father[i]
      Fj<-offspring$Father[j]

      pathA<-(1/4)*coancestry_p[Mi, Mj]
      pathB<-(1/4)*coancestry_p[Fi, Mj]
      pathC<-(1/4)*coancestry_p[Mi, Fj]
      pathD<-(1/4)*coancestry_p[Fi, Fj]

      coancestry_o[i, j]<-(pathA + pathB + pathC + pathD)
      coancestry_o[j, i]<-(pathA + pathB + pathC + pathD)
    }
}

for (i in 1:pop_size){
    coancestry_o[i,i]<-(1/2)*(1 + offspring$IB_coef[i])
}

### Step 11) Save offspring output files every 50 generations for FL day, XY position, neutral genotypes

In [ ]:
if ((g == 1)||(g == 50)||(g == 100)||(g == 150)||(g == 200)||(g == 250)||(g == 300)||(g == 350)||(g == 400)||(g == 450)||(g == 500)){

    offspring_map<-as.data.frame(matrix(nrow=pop_size, ncol=(9+(2*FT_loci)+(2*neut_loci))))
    names(offspring_map)[1:9]<-c("FLday","X_pos","Y_pos","genotypeA","mapA","genotypeB","mapB","genotypeC","mapC")

    offspring_map$FLday<-offspring$FLday
    offspring_map$X_pos<-offspring$X_pos
    offspring_map$Y_pos<-offspring$Y_pos

    for (n in 1:FT_loci)
    {
      names(offspring_map)[10+2*(n-1)]<-paste("loc",n,"a",sep="")
      names(offspring_map)[11+2*(n-1)]<-paste("loc",n,"b",sep="")
    }

    for (i in 1:pop_size){
      for (n in 1:FT_loci)
      {
        offspring_map[i,(10+2*(n-1))]<-offspring[i,(11+2*(n-1))]
        offspring_map[i,(11+2*(n-1))]<-offspring[i,(12+2*(n-1))]
      }
    }

    for (j in 1:neut_loci)
    {
      names(offspring_map)[10+(2*FT_loci)+(2*(j-1))]<-paste("neut",j,"a",sep="")
      names(offspring_map)[11+(2*FT_loci)+(2*(j-1))]<-paste("neut",j,"b",sep="")
    }

    for (i in 1:pop_size){
      for (j in 1:neut_loci)
      {
        offspring_map[i,10+(2*FT_loci)+(2*(j-1))]<-offspring[i,11+(2*FT_loci)+(2*(j-1))]
        offspring_map[i,11+(2*FT_loci)+(2*(j-1))]<-offspring[i,12+(2*FT_loci)+(2*(j-1))]
      }
    }

    for (i in 1:pop_size){
      offspring_map$genotypeA[i]<-paste(offspring$A1[i], offspring$A2[i], sep="")	
      offspring_map$mapA[i]<-if (offspring_map$genotypeA[i]=="aa") {
        0} else if (offspring_map$genotypeA[i]=="AA"){2} else {1}
    }

    for (i in 1:pop_size){
      offspring_map$genotypeB[i]<-paste(offspring$B1[i], offspring$B2[i], sep="")	
      offspring_map$mapB[i]<-if (offspring_map$genotypeB[i]=="bb") {
        0} else if (offspring_map$genotypeB[i]=="BB"){2} else {1}
    }

    for (i in 1:pop_size){
      offspring_map$genotypeC[i]<-paste(offspring$C1[i], offspring$C2[i], sep="")	
      offspring_map$mapC[i]<-if (offspring_map$genotypeC[i]=="cc") {
        0} else if (offspring_map$genotypeC[i]=="CC"){2} else {1}
    }

    write.csv(offspring_map, paste("offspring_map", g, sep="_")
              , row.names=F)

    }
} #end loop over g if

In [ ]:
if ((g == 1)||(g == 50)||(g == 100)||(g == 150)||(g == 200)||(g == 250)||(g == 300)||(g == 350)||(g == 400)||(g == 450)||(g == 500)){

    NH<-subset.data.frame(offspring, select=c(FLday, Mother, Father, X_pos, Y_pos, A1, A2))
    
    #calculate fitness to be used in calculating flowering overlap between individuals
    if(neighbour.fitness == "yes") {
        NH$zFLday<-(NH$FLday-mean(NH$FLday))/sd(NH$FLday) #deviation from mean flowering day
        NH$fitness<-NA
        for (i in 1:pop_size){
            genotype<-paste(NH$A1[i], NH$A2[i], sep="") 
            NH$fitness[i]<-if (genotype=="aa") {
                1 + selection_aa*NH$zFLday[i] } else if (genotype=="AA"){1 + selection_AA*NH$zFLday[i]} else {1 + selection_Aa*NH$zFLday[i]}
        } #sets fitness as values around 1 based on A locus and in what direction of the mean flowering day the individual sits (will be relative because z is calulated relative to population)
    } else (NH$fitness<-1)

    #fill in mom coordinates
    for (m in 1:pop_size) {
      mom.x<-NH$Mother[m]
      NH$M.x[m]<-sapply(X=pos_vector[mom.x], FUN=function(X) {
        as.numeric(strsplit(X, split=",")[[1]][1])
      })
      NH$M.y[m]<-sapply(X=pos_vector[mom.x], FUN=function(X) {
        as.numeric(strsplit(X, split=",")[[1]][2])
      })

    }

    #fill in dad coorindates
    for (p in 1:pop_size) {
      dad.x<-NH$Father[p]
      NH$P.x[p]<-sapply(X=pos_vector[dad.x], FUN=function(X) {
        as.numeric(strsplit(X, split=",")[[1]][1])
      })
      NH$P.y[p]<-sapply(X=pos_vector[dad.x], FUN=function(X) {
        as.numeric(strsplit(X, split=",")[[1]][2])
      })
    }

    #score selfing
    for (k in 1:pop_size) {
      NH$self[k]<-if (NH$Mother[k] == NH$Father[k]) {1} else if (NH$Mother[k] != NH$Father[k]) {0}
    }

    #calculate Euclidean distance between each offspring and parent
    for (m in 1:pop_size){
      NH$mom.dist[m]<-sqrt((NH$M.x[m]-NH$X_pos[m])^2+(NH$M.y[m]-NH$Y_pos[m])^2)
      NH$dad.dist[m]<-sqrt((NH$P.x[m]-NH$X_pos[m])^2+(NH$P.y[m]-NH$Y_pos[m])^2)
    }

    #calculate the mean distance between offspring and parent
    mean.dist<-sum(mean(NH$mom.dist), mean(NH$dad.dist))/2

    #calculate variance of each offspring
    NH<-mutate(NH, ind.var=(mom.dist-mean.dist)^2+(dad.dist-mean.dist)^2)
    
    #create list of distances between offspring and NH
    var.list<-c(NH$mom.dist, NH$dad.dist)
    
    #calculate population variance
    var<-var(var.list)
    
    #create list giving proportion of overlap in flowers between two individuals
    flower.list<-fl_seq/sum(fl_sew)
    flower.list<-cumsum(flower.list)

    #randomly select 100 offspring for which to calculate neighbourhood size
    off.samp<-sample(c(1:pop_size), size=100, replace=FALSE)
    
    #create dataframe to store neighbourhood size for each sampled offspring
    off.df<-as.data.frame(matrix(nrow=100,ncol=1))
    
    #start loop over sampled offspring
    for (w in off.samp){

      #calculate effective density *this effective density DOES NOT include spacing in metres
      if (phenology == "no") {off.df[match(c(w),off.samp),]<- 1} else if (phenology == "yes") {

        #create dataframe
        ind.df<-as.data.frame(matrix(nrow=1, ncol=1))
        ind.df.t<-as.data.frame(matrix(nrow=1, ncol=1))
          
        #get X and Y positions for focal individual
        a<-NH$X_pos[w]
        b<-NH$Y_pos[w]
          
        #get flowering day of focal individual
        FLday.o<-NH$FLday[w]
          
        #get most positive X position on neighbourhood edge
        p.x<-round(sqrt(var))+a #round so can translate to cartesian coordinates
        
        #if p.x is outside the edge of the population, set as highest edge x position (sqrt(pop_size))
        if(p.x > sqrt(pop_size)) {p.x <- sqrt(pop_size)}
          
        #get most negative x position on neighbourhood edge
        n.x<-a-round(sqrt(var)) #round so can translate to cartesian coordinates
          
        #if n.x is outside the edge of the population, set as lowest x position (1)
        if(n.x < 1) {n.x <- 1}
          
        #begin loop over range of x positions describing the genetic neighbourhood
        for (x in n.x:p.x){
            
          #get most positive Y position on neighbourhood edge
          p.y<-if (var-(x-a)^2 < 0) {b} else {round(sqrt(var-(x-a)^2))+b}
            
          #if p.y is outside the edge of the population, set as highest y position (sqrt(pop_size))
          if(p.y > sqrt(pop_size)) {p.y <- sqrt(pop_size)}
            
          #get most negative Y position on neighbourhood edge
          n.y <- if (var-(x-a)^2 < 0) {b} else {b-round(sqrt(var-(x-a)^2))}
            
          #if n.y is outside the edge of the population, set as lowest y position (1)
          if(n.y < 1) {n.y <- 1}
            
          #start loop over range of y positions for a given possible x position of the genetic neighbourhood
          for (z in n.y:p.y) {
              
            #find individual occupying space given by X_pos x and Y_pos z
            NH.temp<-filter(NH, X_pos==x, Y_pos==z)
              
            #get number of days overlapping in flowering schedule between focal individual and NH.temp individual
            FLday.t<-abs(NH.temp$FLday[1]-FLday.o) #paired indivdual relative flowering day
              
            #set up integral for overlap in flowering schedule
            FLday.t<-abs(NH.temp$FLday[1]-FLday.o) #number of days overlapping
            
            #if there is overlap in flowering schedules, get proportion of total flowers produced that overlap
            if (FLday.t <= 14) {
                flower.over.t<-flower.list[15-FLday.t]
            }
             
            #adjust proportion by fitness of paired individual
            if(neigbour.fitness == "yes") {
                flower.over.t<-flower.over.t*NH.temp$fitness
            }
            
            #store proportion in dataframe describing focal individual's neighbourhood
            ind.df.t[1,1]<-flower.over.t
            ind.df<-bind_rows(ind.df, ind.df.t)
              
          } #next y position
        } #next x position
          
        off.df[match(c(w),off.samp),]<-sum(ind.df[,1], na.rm=TRUE)/length(ind.df[,1])
          
      } #end if/else
    } #next offspring

    write.csv(off.df, paste("off.df.", g, ".csv", sep=""))
    write.csv(NH, paste("NH.", g, ".csv", sep=""))

    #calculate and store size of neigbourhood based on variance and average effective density
    sum.N[g,1]<-(4*3.14*var*mean(off.df$V1, na.rm=TRUE))
        
    #store variance
    sum.N[g,2]<-var
        
    #store effective density
    sum.N[g,3]<-mean(off.df$V1, na.rm=TRUE)
        
    #store selfing rate
    self.df[g,1]<-mean(NH$self)

    #write csv files with neighbourhood and selfing information
    write.csv(sum.N,
              "sum.N.csv", row.names=F)
    write.csv(self.df, 
              "self.df.csv", row.names=F)
} #end loop over g if

### If g is less than no_gen, set offspring as parents and repeat

In [ ]:
if (g<no_gen){
    offspring->parents
    coancestry_o->coancestry_p}
}